In [5]:
import cv2
import numpy as np
import pandas as pd
import os

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import keras
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications import VGG16,ResNet101,ResNet101V2, InceptionResNetV2,NASNetLarge,Xception

In [6]:
cd /

/


In [28]:
cd kaggle/input/hackerearth-deep-learning-identify-the-snake-breed/dataset

/kaggle/input/hackerearth-deep-learning-identify-the-snake-breed/dataset


In [13]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [14]:
def addext(nm):
    return nm+'.jpg'

In [16]:
train['image_id'] = train['image_id'].apply(addext)

In [15]:
test['image_id'] = test['image_id'].apply(addext)

In [17]:
datagen = ImageDataGenerator(rescale=1/255,
        #validation_split=0.10,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 

In [18]:
train_datagen = datagen.flow_from_dataframe(
        dataframe=train,
        directory="./train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=54,
        seed=42,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224))

Found 5508 validated image filenames belonging to 35 classes.


In [ ]:
valid_datagen = datagen.flow_from_dataframe(
        dataframe=train,
        directory="./train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=54,
        seed=42,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224))

In [19]:
#base_model = EfficientNetB7(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = ResNet101(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
base_model = ResNet101V2(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = Xception(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = NASNetLarge(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')
#base_model = InceptionResNetV2(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')

#base_model.trainable = False
for layer in base_model.layers[:-3]:
    layer.trainable = False
for layer in base_model.layers[-3:]:
    layer.trainable = True
model = Sequential([
  base_model, 
  Dense(1024, activation='relu'),
  Dropout(0.3),
  Dense(512,activation = 'relu'),
  Dropout(0.3),
  Dense(256, activation='relu'),
  Dropout(0.3),
  Dense(128, activation = 'relu'),
  Dense(35, activation='softmax'),
])

171319296/171317808 [==============================] - 6s 0us/step


In [20]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.0008, decay = 1e-7)
model.compile(optimizer=optimizer,loss ="categorical_crossentropy", metrics=["accuracy"])

In [29]:
model.fit(train_datagen,epochs=20, 
                    #validation_data=valid_datagen, validation_steps=20,
                    steps_per_epoch=81)

Epoch 1/20
81/81 [==============================] - 54s 665ms/step - loss: 1.7980 - accuracy: 0.4979
Epoch 2/20
81/81 [==============================] - 54s 665ms/step - loss: 1.8012 - accuracy: 0.4947
Epoch 3/20
81/81 [==============================] - 54s 661ms/step - loss: 1.7662 - accuracy: 0.5096
Epoch 4/20
81/81 [==============================] - 53s 651ms/step - loss: 1.7766 - accuracy: 0.4911
Epoch 5/20
81/81 [==============================] - 53s 656ms/step - loss: 1.8031 - accuracy: 0.4936
Epoch 6/20
81/81 [==============================] - 53s 657ms/step - loss: 1.7857 - accuracy: 0.4947
Epoch 7/20
81/81 [==============================] - 52s 646ms/step - loss: 1.7683 - accuracy: 0.5034
Epoch 8/20
81/81 [==============================] - 54s 661ms/step - loss: 1.7527 - accuracy: 0.5062
Epoch 9/20
81/81 [==============================] - 53s 655ms/step - loss: 1.7533 - accuracy: 0.5085
Epoch 10/20
81/81 [==============================] - 52s 648ms/step - loss: 1.7071 - accura

In [22]:
test_df=pd.read_csv('test.csv')
test_df["image_id"]=test_df["image_id"].apply(addext)
test_datagen = ImageDataGenerator(rescale=1./255,)
test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                            directory="test/",
                                            x_col="image_id",
                                            y_col=None,
                                            batch_size=128,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(331,331))


Found 2361 validated image filenames.


In [30]:
y_pred=model.predict_generator(test_generator,verbose=1)
y_pred_2=np.argmax(y_pred,axis=1)
labels = (train_datagen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]
test_df=pd.read_csv('test.csv')
test_name=test_df['image_id'].values
data=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])

19/19 [==============================] - 13s 686ms/step


In [31]:
cd /

/


In [32]:
cd kaggle/working

/kaggle/working


In [26]:
data.to_csv('resnet101V2.csv',index = False)

In [27]:
cd /

/
